In [1]:
# Importing necessary modules

import sklearn.datasets as datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import pandas as pd 
# read text file into pandas DataFrame and 
# create header 
import os
Column_names = ["TIME_str", "TIME_sec", "latitude", "longitude", "ALTITUDE_FT", "GROUNDTRACK_DEG", \
            "GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN", "Vertical_label",\
           "Horizontal_label", "Speed_label", "Turn_rate_deg_per_sec", "REL_HEADING", "DIST_TO_DEST", "MANEUVER_ID" ]
Selected_col_names = ["GROUNDSPEED_KTS", "VERTICALSPEED_FT_PER_MIN",  \
                      "DIST_TO_DEST"]
Target_category_name = "Vertical_label"
data_name = "HOLD_3"
cwd = os.getcwd()
mypath = cwd + f"/../data/{data_name}/"
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
count = 0

for file_name in onlyfiles:
    if "Traj" in file_name:
        if count == 0:
            file_path = mypath + file_name
            df = pd.read_csv(file_path, sep=",", header=None)
            df.columns = Column_names
            X_df = df[Selected_col_names]
            Y_target = df[Target_category_name]
        else:
            file_path = mypath + file_name
            df_temp = pd.read_csv(file_path, sep=",", header=None)
            df_temp.columns = Column_names
            
            X_df_temp = df_temp[Selected_col_names]
            Y_target_temp = df_temp[Target_category_name]
            X_df = pd.concat([X_df, X_df_temp])
            Y_target = pd.concat([Y_target, Y_target_temp])
        count +=1
        # if count >= 10:
        #     break

In [4]:
X_df["GROUNDSPEED_KTS"].max()

547.0

In [5]:
X_df["GROUNDSPEED_KTS"].min()

0.0

In [6]:
target_names =  list(set(list(Y_target)))
word_map_to_num = set(list(Y_target))
print("category:")
word_map_to_num


category:


{'  Over 250 kts under 10000ft',
 ' Climb_Nominal',
 ' Descend_Nominal',
 ' MaintainCurrentAltitude',
 ' No Conclusion: Vertical'}

In [7]:
n_sample = len(X_df)
print("number of points:", n_sample)
X_df = X_df.head(n_sample)
Y_target = Y_target.head(n_sample)
Y = []
for y in Y_target:
    for idx, s in enumerate(word_map_to_num):
        if s == y:
            Y.append(idx)
Y = np.array(Y)

number of points: 54570


In [8]:
import pandas as pd 
import dtreeviz
from sklearn.tree import export_text
# read text file into pandas DataFrame and 
# create header 
import os
cwd = os.getcwd()

# file_name_test = "Traj_AAL80-11121269.dat" # worst
file_name_test = "Traj_AAL245-11147798_L_HOLD.dat" # normal

file_path_test = mypath + file_name_test


# file_name_test = "Traj_AAL1252-11172342POS_HOLD.dat"

# file_path_test = cwd + "/../data/HOLD/" + file_name_test


df_test = pd.read_csv(file_path_test, sep=",", header=None)
df_test.columns = Column_names
X_df_test = df_test[Selected_col_names]

In [9]:
print("number of testing points:", len(X_df_test))

number of testing points: 3407


In [10]:
Z_target = df_test[Target_category_name]
n_sample = len(X_df_test)
X_df_test = X_df_test.head(n_sample)
Z_target = Z_target.head(n_sample)
Z = []
for z in Z_target:
    for idx, s in enumerate(word_map_to_num):
        if s == z:
            Z.append(idx)
Z = np.array(Z)
len(Z)

3407

## target 1: vertical_label

# Entropy

In [11]:
# Entropy, small tree with train data

criterion = "entropy"
max_depth = 8
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg

0.8870075132856882


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


In [31]:
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 282.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -736.00
|   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -736.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -928.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 235.50
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 4.02
|   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |--- DIST_TO_DEST >  4.02
|   |   |   |   |   |   |   |   |--- class:  Descend_Nomin

In [12]:
# Entropy, small tree with train data
from sklearn.tree import export_text
from sklearn.tree import export_text_in_conditions

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
viz_model = dtreeviz.model(dtree_small_entropy, X_df, Y,
                target_name="target",
                feature_names=list(X_df.columns),
                class_names=target_names)

v = viz_model.view()     # render as SVG into internal object 
v                 # pop up window
v.save(f"{data_name}_all_{Target_category_name}_{criterion}_without_xy_" + f"max_depth:{max_depth}"  + ".svg")  # optionally save as svg
r = export_text_in_conditions(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)

0.8870075132856882


/Users/binshuaiwang/PycharmProjects/XAI_project/venv/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names


If VERTICALSPEED_FT_PER_MIN <= -160.00 && VERTICALSPEED_FT_PER_MIN <= -672.00 && DIST_TO_DEST <= 96.90 && GROUNDSPEED_KTS <= 282.50 && GROUNDSPEED_KTS <= 174.50 && VERTICALSPEED_FT_PER_MIN <= -736.00, then  class:  Descend_Nominal

If VERTICALSPEED_FT_PER_MIN <= -160.00 && VERTICALSPEED_FT_PER_MIN <= -672.00 && DIST_TO_DEST <= 96.90 && GROUNDSPEED_KTS <= 282.50 && GROUNDSPEED_KTS <= 174.50 && VERTICALSPEED_FT_PER_MIN > -736.00 && GROUNDSPEED_KTS <= 163.00, then  class:  Descend_Nominal

If VERTICALSPEED_FT_PER_MIN <= -160.00 && VERTICALSPEED_FT_PER_MIN <= -672.00 && DIST_TO_DEST <= 96.90 && GROUNDSPEED_KTS <= 282.50 && GROUNDSPEED_KTS <= 174.50 && VERTICALSPEED_FT_PER_MIN > -736.00 && GROUNDSPEED_KTS > 163.00, then  class:  Descend_Nominal

If VERTICALSPEED_FT_PER_MIN <= -160.00 && VERTICALSPEED_FT_PER_MIN <= -672.00 && DIST_TO_DEST <= 96.90 && GROUNDSPEED_KTS <= 282.50 && GROUNDSPEED_KTS > 174.50 && VERTICALSPEED_FT_PER_MIN <= -928.00 && GROUNDSPEED_KTS <= 235.50 && DIST_TO_DEST <= 4.

In [27]:
ss = dict()
tt = "dd"
ss[tt] = 1

if "ss" not in ss:
    ss["ss"] = 1
ss2 = dict()
set(ss2.keys())

set()

In [28]:
ss

{'dd': 1, 'ss': 1}

In [21]:
for s in ss.keys():
    print(s)

ss


In [13]:
from sklearn.tree import export_text_in_words

r = export_text_in_words(X_df, dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)

If VERTICALSPEED_FT_PER_MIN is high, and VERTICALSPEED_FT_PER_MIN is high, and DIST_TO_DEST is extremely close, and GROUNDSPEED_KTS is normal, and GROUNDSPEED_KTS is low, and VERTICALSPEED_FT_PER_MIN is high, then  class:  Descend_Nominal

If VERTICALSPEED_FT_PER_MIN is high, and VERTICALSPEED_FT_PER_MIN is high, and DIST_TO_DEST is extremely close, and GROUNDSPEED_KTS is normal, and GROUNDSPEED_KTS is low, and VERTICALSPEED_FT_PER_MIN is high, and GROUNDSPEED_KTS is low, then  class:  Descend_Nominal

If VERTICALSPEED_FT_PER_MIN is high, and VERTICALSPEED_FT_PER_MIN is high, and DIST_TO_DEST is extremely close, and GROUNDSPEED_KTS is normal, and GROUNDSPEED_KTS is low, and VERTICALSPEED_FT_PER_MIN is high, and GROUNDSPEED_KTS is low, then  class:  Descend_Nominal

If VERTICALSPEED_FT_PER_MIN is high, and VERTICALSPEED_FT_PER_MIN is high, and DIST_TO_DEST is extremely close, and GROUNDSPEED_KTS is normal, and GROUNDSPEED_KTS is low, and VERTICALSPEED_FT_PER_MIN is normal, and GROUNDSPE

In [14]:
import graphviz
from sklearn.tree import export_graphviz
dot_data = export_graphviz(X_df, dtree_small_entropy,
                           feature_names=X_df.columns,  
                           class_names = target_names,
                           filled=False,  
                           rounded = True,
                            precision = 2,
                           proportion=False)
graph = graphviz.Source(dot_data)
graph.format = "png"
graph.render(f"{data_name}__all_{Target_category_name}_{criterion}" + "importances")

'HOLD_3__all_Vertical_label_entropyimportances.png'

In [15]:
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)

In [16]:
print(r)

|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 282.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -736.00
|   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -736.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -928.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 235.50
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 4.02
|   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |--- DIST_TO_DEST >  4.02
|   |   |   |   |   |   |   |   |--- class:  Descend_Nomin

In [17]:
tree = model.tree_

In [18]:
print(tree.children_left[tree.children_left[2]])

4


In [19]:
criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9028770386659337
|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 282.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -736.00
|   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -736.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -928.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 235.50
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 4.02
|   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |--- DIST_TO_DEST >  4.02
|   |   |   |   |   |   |   |   |--- DI

In [20]:
# Entropy, large tree with train data

criterion = "entropy"
dtree_large_n = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_n.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_n, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9981308411214954
|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 282.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -736.00
|   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -736.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  163.00
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 6.20
|   |   |   |   |   |   |   |   |--- DIST_TO_DEST <= 5.30
|   |   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |   |--- DIST_TO_DEST >  5.30
|   |   |   |   |   |   |   |   |   |--- class:  No Conclusion: Vertical
|   |   |   |   |   |   |   |--- DIST_TO_DEST >  6.20
|   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |

In [21]:
# Entropy, large tree with test data

criterion = "entropy"
dtree_large_entropy = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9999999999999999
|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 282.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -736.00
|   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -736.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  163.00
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 6.20
|   |   |   |   |   |   |   |   |--- DIST_TO_DEST <= 5.30
|   |   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |   |--- DIST_TO_DEST >  5.30
|   |   |   |   |   |   |   |   |   |--- class:  No Conclusion: Vertical
|   |   |   |   |   |   |   |--- DIST_TO_DEST >  6.20
|   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |

In [22]:
# Entropy, small tree with test data

criterion = "entropy"
dtree_small_entropy = DecisionTreeClassifier(criterion=criterion, max_depth=10, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_entropy, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8652773701203404
|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 282.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -736.00
|   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -736.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |--- GROUNDSPEED_KTS >  163.00
|   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  174.50
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -928.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 235.50
|   |   |   |   |   |   |   |--- DIST_TO_DEST <= 4.02
|   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |--- DIST_TO_DEST >  4.02
|   |   |   |   |   |   |   |   |--- DI

In [23]:
# Entropy, large rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9856178456119753


In [24]:
# Entropy, small rf with test data

criterion = "entropy"
num_of_trees = 3
rf_small_entropy = RandomForestClassifier(n_estimators=num_of_trees, max_depth=5, min_samples_leaf=20)
model=rf_small_entropy.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8652773701203404


# Gini

In [25]:
# Gini, small tree with train data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8529228513835441
|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -480.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 283.50
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -672.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |--- GROUNDSPEED_KTS >  283.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 308.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  308.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |--- DIST_TO_DEST >  96.90
|   |   |   |--- DIST_TO_DEST <= 295.01
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -928.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -928.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |--- DIST_TO_DEST >  295.01
|   |   |   |   |--- class:  Descend_Nomina

In [26]:
# Gini, large tree with train data
criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9981308411214954
|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -480.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 283.50
|   |   |   |   |--- DIST_TO_DEST <= 54.39
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -736.00
|   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -736.00
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 163.00
|   |   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  163.00
|   |   |   |   |   |   |   |   |   |--- DIST_TO_DEST <= 6.20
|   |   |   |   |   |   |   |   |   |   |--- DIST_TO_DEST <= 5.30
|   |   |   |   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |   |   |   |--- DIST_TO_DEST >  5.30
|

In [27]:
# Gini, large tree with test data

criterion = "gini"
dtree_large_gini = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_large_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.9999999999999999
|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -480.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 283.50
|   |   |   |   |--- DIST_TO_DEST <= 54.39
|   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -736.00
|   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -736.00
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS <= 163.00
|   |   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |   |--- GROUNDSPEED_KTS >  163.00
|   |   |   |   |   |   |   |   |   |--- DIST_TO_DEST <= 6.20
|   |   |   |   |   |   |   |   |   |   |--- DIST_TO_DEST <= 5.30
|   |   |   |   |   |   |   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |   |   |   |   |   |   |--- DIST_TO_DEST >  5.30
|

In [28]:
# Gini, small tree with test data

criterion = "gini"
dtree_small_gini = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_gini.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_gini, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8643968300557675
|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -480.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 283.50
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -672.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |--- GROUNDSPEED_KTS >  283.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 308.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  308.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |--- DIST_TO_DEST >  96.90
|   |   |   |--- DIST_TO_DEST <= 295.01
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -928.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -928.00
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |--- DIST_TO_DEST >  295.01
|   |   |   |   |--- class:  Descend_Nomina

## Log_loss

In [29]:
# Log_loss, small tree with train data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)
r = export_text(dtree_small_Log_loss, feature_names=list(X_df.columns), class_names=target_names, show_weights=False)
print(r)

0.8629649990837457
|--- VERTICALSPEED_FT_PER_MIN <= -160.00
|   |--- VERTICALSPEED_FT_PER_MIN <= -672.00
|   |   |--- DIST_TO_DEST <= 96.90
|   |   |   |--- GROUNDSPEED_KTS <= 282.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 174.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  174.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |--- GROUNDSPEED_KTS >  282.50
|   |   |   |   |--- GROUNDSPEED_KTS <= 316.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  316.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |--- DIST_TO_DEST >  96.90
|   |   |   |--- VERTICALSPEED_FT_PER_MIN <= -928.00
|   |   |   |   |--- GROUNDSPEED_KTS <= 342.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |   |--- GROUNDSPEED_KTS >  342.50
|   |   |   |   |   |--- class:  Descend_Nominal
|   |   |   |--- VERTICALSPEED_FT_PER_MIN >  -928.00
|   |   |   |   |--- GROUNDSPEED_KTS <= 405.00
|   |   | 

In [30]:
# Log_loss, large tree with train data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df))
diff = res - Y
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9981308411214954


In [31]:
# Log_loss, large tree with test data
criterion = "log_loss"
dtree_large_Log_loss = DecisionTreeClassifier(criterion=criterion)   # (random_state=1234)
model=dtree_large_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.9999999999999999


In [32]:
# Log_loss, small tree with test data

criterion = "log_loss"
dtree_small_Log_loss = DecisionTreeClassifier(criterion=criterion, max_depth=5, min_samples_leaf=20)   # (random_state=1234)
model=dtree_small_Log_loss.fit(X_df,Y)
res = np.array(model.predict(X_df_test))
diff = res - Z
accuracy = 1/len(res) * sum([1 for x in diff if x == 0])
print(accuracy)

0.8646903434106251
